In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
test =  pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')
sub =  pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
# Check for Null values
train.isnull().any()

In [ ]:
# Lets check about datatype of all attributes
train.info()

In [ ]:
# Id column does not have much to contribute, so we'll drop it
train.drop('id', axis = 1, inplace = True)

In [ ]:
# lets get some training data
train.head()

In [ ]:
# Since, columns 1 to 10 are of dtype - object(A, B, C, D, E..), we'll use encoding to make them looklike the rest of the column values

categorical_cols=['cat'+str(i) for i in range(10)]
continous_cols=['cont'+str(i) for i in range(14)]

In [ ]:
from sklearn.preprocessing import LabelEncoder
for e in categorical_cols:
    le = LabelEncoder()
    train[e]=le.fit_transform(train[e])
    test[e]=le.transform(test[e])

In [ ]:
data=train[categorical_cols+continous_cols]

In [ ]:
X = data
y = train['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
import xgboost as xg

xgb_reg = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = 10, seed = 123)
xgb_reg.fit(X_train, y_train)

prediction = xgb_reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(prediction, y_test)
mae = mean_absolute_error(prediction, y_test)

print('mse - ', mse, 'mae - ',mae)

In [ ]:
test_prediction = xgb_reg.predict(test.drop('id', axis = 1))
test_prediction


--------------------------------------------------------------------------------

In [ ]:
pred = xgb_reg.predict(test.drop('id', axis = 1))
output = pd.DataFrame({'id':test['id'],'target':pred})
output.set_index('id',inplace=True)
output.to_csv('output.csv')

In [ ]:
output.head()

In [ ]:
sub['target']=pred
sub.to_csv('submission.csv', index=False)